# Grafice raport CDOS

Acest Jupyter Notebook permite generarea unor grafice pe baza răspunsurilor la chestionarul CDOS.

## Import de biblioteci

In [ ]:
from pathlib import Path
import re
from textwrap import fill

import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams

## Configurare folder de output

Toate graficele generate se vor salva în acest loc.

In [ ]:
output_dir = Path('./output/')
output_dir.mkdir(parents=True, exist_ok=True)

## Configurare stilizare

In [ ]:
# Creștem densitatea în pixeli a figurilor generate
rcParams['figure.dpi'] = 96

# TODO: nu-mi dau seama de ce nu merge să schimbăm fontul
# https://fonts.google.com/specimen/IBM+Plex+Sans?query=plex
# rcParams['font.family'] = 'sans-serif'
# rcParams['font.sans-serif'] = ['IBM Plex Sans']

# Set the default style to be used in making graphics
plt.style.use('default')

## Citire date

In [ ]:
df = pd.read_excel('./CDOS Licență (Responses).xlsx', engine='openpyxl')

## Generare grafice

### Mapare indici coloane

Studenții aleg inițial anul și specializarea lor. În funcție de acestea, sunt redirecționați la secțiuni diferite din formular.

Acest bloc de cod definește numărul coloanei de la care încep coloanele corespunzătoare fiecărei specializări.

In [ ]:
# Dictionary mapping each (year, specialization) pair to a column number in the Excel
offset_map = {
    'I': {
        'Informatică - seria 13': 5,
        'Informatică - seria 14': 5,
        'Informatică - seria 15': 5,
        'Matematică - seria 10': 49,
        'Matematică - seria 11': 49,
        'Calculatoare și tehnologia informației': 153,
    },
    'II': {
        'Informatică - seria 23': 19,
        'Informatică - seria 24': 19,
        'Matematici Aplicate': 124,
        'Matematică': 64,
        'Matematică - Informatică': 94,
        'Calculatoare și Tehnologia Informației': 167,
    },
    'III': {
        'Informatică - seria 33': 33,
        'Informatică - seria 34': 33,
        'Matematici Aplicate': 138,
        'Matematică - Informatică': 108,
        'Calculatoare și Tehnologia Informației': 181,
    },
    'IV': {
        'Calculatoare și Tehnologia Informației': 195        
    }
}

### Întrebări pe specializări

In [ ]:
# Acest regex e folosit ca să elimine sufixul numeric
# adăugat automat de pandas pe coloanele cu nume duplicat.
#
# Ex. dacă coloana inițială se numea "test", puteam avea
# o coloană duplicată redenumită la "test.13".
remove_column_suffix = re.compile(r'\.[0-9]+$')

# Trebuie să luăm fiecare an de studiu pe rând
for index, year in enumerate(('I', 'II', 'III', 'IV')):
    # Filtrăm răspunsurile
    df_year = df[df['Anul de studiu în 2020-2021'] == f'Anul {year}'].copy()

    # Determinăm specializarea
    if year == 'IV':
        # Pentru anul 4 nu avem o coloană de "Specializare",
        # pentru că singura posibilitate ar fi fost CTI.
        #
        # Așa că o adăugăm manual.
        specialization_column_name = 'Specializare anul IV'
        df_year[specialization_column_name] = 'Calculatoare și Tehnologia Informației'
    else:
        # Coloana care indică specializarea e diferită în funcție de an
        specialization_column_index = 2 + index
        specialization_column_name = df_year.columns[specialization_column_index]
    
    specialization = df_year[specialization_column_name]
    
    print('Anul', year, f'({len(df_year)} respondenți)')
    
    # Grupăm răspunsurile pe ani după specializare
    for group_name, df_group in df_year.groupby(specialization):
        print('-', group_name)
        offset = offset_map[year][group_name]
        
        # Determin directorul
        directory_path = output_dir / f'Anul {year}' / group_name
        directory_path.mkdir(parents=True, exist_ok=True)

        for i in range(3):
            index = offset + i

            column_name = df_group.columns[index]
            values = df_group[column_name]
            
            # Sparg coloana de top 3 materii (un șir de alegeri separate prin virgulă)
            # în 3 coloane distincte.
            #
            # Bazat pe https://stackoverflow.com/a/46856366/5723188
            values = values.str.split(',', expand=True)
            
            # Determin frecvența cu care apare pe fiecare nouă coloană fiecare materie.
            #
            # Bazat pe https://stackoverflow.com/a/32589877/5723188
            values = values.apply(pd.Series.value_counts)
            
            # Determin per total cât de frecvent a apărut fiecare materie la această întrebare.
            values = values.sum(axis=1)
            
            # Sortez după numărul de apariții
            values = values.sort_values()

            # Plotez graficul
            plt.figure()
            question = remove_column_suffix.sub('', column_name)
            plt.suptitle(question)
            values.plot.barh()
            plt.savefig(directory_path / f'{i}.png', bbox_inches='tight')
            plt.close()

## Pie chart

In [ ]:
column = df.columns[-6]
answers = df[column]
answers = answers.fillna('Nu știu')

counts = answers.value_counts(dropna=False)

counts = counts[['Da', 'Nu', 'Nu știu']]

counts.plot.pie(colormap='plasma', ylabel='', shadow=True,
                # TODO: nu se prea vede bine textul generat automat
                #autopct='%1.1f%%',
                explode=(0, 0.2, 0))
plt.suptitle(fill(column, 60))
plt.text(0, -1.3, f'{answers.count()} respondenți', ha='center', va='center')
plt.show()